In [122]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, Reshape, Concatenate, Flatten, Dense, Dropout
from tensorflow.keras import Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [137]:
class FFN(Model):
    def __init__(self, num_filters=8, kernel_size=1):
        super(FFN, self).__init__()
        self.num_filters = num_filters
        self.kernel_size = kernel_size


        self.conv1_layers = Conv1D(num_filters, kernel_size, dilation_rate=1, activation='relu') 
        self.conv2_layers = Conv1D(num_filters, kernel_size, dilation_rate=2, activation='relu') 
        self.conv3_layers = Conv1D(num_filters, kernel_size, dilation_rate=4, activation='relu')        
        # self.conv1_layers = Conv1D(num_filters, kernel_size, activation='relu') 
        # self.conv2_layers = Conv1D(num_filters, kernel_size, activation='relu') 
        # self.conv3_layers = Conv1D(num_filters, kernel_size, activation='relu')
        # 
        # Define dense layers for the MLP
        self.dense1 = Dense(128, activation='relu')
        self.dropout1 = Dropout(0.2)
        self.dense2 = Dense(64, activation='relu')
        self.dropout2 = Dropout(0.2)
        self.dense3 = Dense(32, activation='relu')
        self.dropout3 = Dropout(0.2)
        self.output_layer = Dense(1, activation='linear')

    def process_feature(self, input_feature):
        conv1 = self.conv1_layers(input_feature)
        conv2 = self.conv2_layers(input_feature)
        conv3 = self.conv3_layers(input_feature)
        
        context_length = 24
        conv1_reshaped = Reshape((context_length, self.num_filters))(conv1)
        conv2_reshaped = Reshape((context_length, self.num_filters))(conv2)
        conv3_reshaped = Reshape((context_length, self.num_filters))(conv3)
        
        concatenated = Concatenate(axis=2)([conv1_reshaped, conv2_reshaped, conv3_reshaped])
        
        return concatenated
    
    @tf.function 
    def call(self, inputs, training=False):
        # print(f"inputs {inputs.shape}")
        # Split the input into separate features
        features = [Reshape((24, 1))(inputs[:, :, i]) for i in range(6)]
        
        # print(f"features {features}")
        
        

        # Process each feature separately
        processed_features = [self.process_feature(feature) for feature in features]

        # Concatenate the processed features along the time dimension
        concatenated_features = Concatenate(axis=1)(processed_features)

        # Flatten the concatenated features
        flattened_features = Flatten()(concatenated_features)

        # Dense layers (MLP)
        x = self.dense1(flattened_features)
        # x = self.dropout1(x, training=training)
        x = self.dense2(x)
        # x = self.dropout2(x, training=training)
        x = self.dense3(x)
        # x = self.dropout3(x, training=training)

        # Output layer
        output = self.output_layer(x)
        
        return output


In [138]:
# data = pd.read_csv('data/ETTh2_head.csv')
data = pd.read_csv('data/ETTh2.csv')
target_col = 'OT'
X = data[['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL']].values

Y = data[target_col].values

In [148]:
def create_sliding_windows(data, target, input_window_size, output_window_size):
    X = []
    Y = []
    for i in range(len(data) - input_window_size - output_window_size + 1):
        X.append(data[i:i+input_window_size])
        Y.append(target[i+input_window_size:i+input_window_size+output_window_size])
    return np.array(X), np.array(Y)


# Create sliding windows
input_window_size = 24
output_window_size = 1

x,y = create_sliding_windows(X, Y, input_window_size, output_window_size)


X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                    train_size=0.8,
                                                    random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    train_size=0.8,
                                                    random_state=1)
# 
# 
# X_train, Y_train = create_sliding_windows(X_train, y_train, input_window_size, output_window_size)
# 
# X_val, y_val = create_sliding_windows(X_val, y_val, input_window_size, output_window_size)
# 
# X_test, y_test = create_sliding_windows(X_test, y_test, input_window_size, output_window_size)

print(x.shape)  # (num_samples, input_window_size, num_features)
print(y.shape)  # (num_samples, output_window_size)


(17396, 24, 6)
(17396, 1)


In [149]:
# Define the input shape
input_shape = (None, input_window_size, X_train.shape[2])

# Create an instance of the model
model = FFN()

# Build the model by passing a dummy input through it
model.build(input_shape)

# Print the model summary


# Compile the model
model.compile(optimizer='adam', loss='mse')
model.summary()
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(x=X_train,
        y=y_train,
        batch_size=64,
        callbacks=[early_stopping],
        epochs=10000,
        validation_data=[X_val, y_val])
        

Model: "ffn_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_250 (Conv1D)         multiple                  16        
                                                                 
 conv1d_251 (Conv1D)         multiple                  16        
                                                                 
 conv1d_252 (Conv1D)         multiple                  16        
                                                                 
 dense_142 (Dense)           multiple                  442496    
                                                                 
 dropout_102 (Dropout)       multiple                  0 (unused)
                                                                 
 dense_143 (Dense)           multiple                  8256      
                                                                 
 dropout_103 (Dropout)       multiple                  0 (un

2024-07-02 18:17:37.987575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,24,6]
	 [[{{node Placeholder}}]]
2024-07-02 18:17:38.183461: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/conv1d_251/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/conv1d_251/Conv1D/BatchToSpaceND/block_shape' with dtype int32 and shape [1]
	 [[{{node gradients/conv1d_251/Conv1D/BatchToSpaceND_grad/SpaceToBatchND/conv1d_251/Conv1D/BatchToSpaceND/block_shape}}]]
2024-07-02 18:17:38.183504: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (th

174/174 [==============================] - 1s 4ms/step - loss: 95.9965 - val_loss: 76.3056
Epoch 2/10000
174/174 [==============================] - 1s 4ms/step - loss: 71.5280 - val_loss: 68.2309
Epoch 3/10000
174/174 [==============================] - 1s 4ms/step - loss: 66.7826 - val_loss: 68.9249
Epoch 4/10000
174/174 [==============================] - 1s 4ms/step - loss: 62.4172 - val_loss: 61.6057
Epoch 5/10000
174/174 [==============================] - 1s 4ms/step - loss: 58.3630 - val_loss: 52.8242
Epoch 6/10000
174/174 [==============================] - 1s 4ms/step - loss: 57.1468 - val_loss: 52.8728
Epoch 7/10000
174/174 [==============================] - 1s 4ms/step - loss: 49.8641 - val_loss: 49.1365
Epoch 8/10000
174/174 [==============================] - 1s 4ms/step - loss: 48.5274 - val_loss: 48.2621
Epoch 9/10000
174/174 [==============================] - 1s 4ms/step - loss: 48.5161 - val_loss: 45.3892
Epoch 10/10000
174/174 [==============================] - 1s 5ms/step

KeyboardInterrupt: 